#### 这个版本的tensorflow 由于时间过于长远与本地cuda 版本不兼容（12.1）,没有tensorflow.examples这个attribute,所以下面给了一个重构并且将模型封装的版本，适用于tensorflow==2.12.0

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

learning_rate = 1e-4
keep_prob_rate = 0.7 # 
max_epoch = 2000
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    # 每一维度  滑动步长全部是 1， padding 方式 选择 same
    # 提示 使用函数  tf.nn.conv2d
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')



def max_pool_2x2(x):
    # 滑动步长 是 2步; 池化窗口的尺度 高和宽度都是2; padding 方式 请选择 same
    # 提示 使用函数  tf.nn.max_pool
    
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 784])/255.
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 28, 28, 1])

#  卷积层 1
## conv1 layer ##
# patch 7x7, in size 1, out size 32
W_conv1 = weight_variable([7, 7, 1, 32])   
b_conv1 = b_conv1 = bias_variable([32])                     
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)                    # 卷积  自己选择 选择激活函数
h_pool1 = max_pool_2x2(h_conv1)  # 池化               

# 卷积层 2
W_conv2 = weight_variable([5, 5, 32, 64])                 # patch 5x5, in size 32, out size 64
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)                       # 卷积  自己选择 选择激活函数
h_pool2 = max_pool_2x2(h_conv2)                       # 池化

#  全连接层 1
## fc1 layer ##
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 全连接层 2
## fc2 layer ##
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


# 交叉熵函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for i in range(max_epoch):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob:keep_prob_rate})
        if i % 100 == 0:
            print(compute_accuracy(
                mnist.test.images[:1000], mnist.test.labels[:1000]))


ModuleNotFoundError: No module named 'tensorflow.examples'

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential

# Load  MNIST dataset
(mnist_train_images, mnist_train_labels), (mnist_test_images, mnist_test_labels) = mnist.load_data()
mnist_train_images, mnist_test_images = mnist_train_images / 255.0, mnist_test_images / 255.0

# 变成 one-hot encoding
mnist_train_labels = to_categorical(mnist_train_labels, 10)
mnist_test_labels = to_categorical(mnist_test_labels, 10)

# 封装model
model = Sequential([
    Conv2D(32, kernel_size=(7, 7), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (5, 5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# compile
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train
model.fit(mnist_train_images.reshape(-1, 28, 28, 1), mnist_train_labels, batch_size=100, epochs=2000//100, validation_split=0.1)

# Evaluate
test_loss, test_acc = model.evaluate(mnist_test_images.reshape(-1, 28, 28, 1), mnist_test_labels, verbose=2)

print('\nTest accuracy:', test_acc)


Epoch 1/20
540/540 [==============================] - 13s 24ms/step - loss: 0.1991 - accuracy: 0.9372 - val_loss: 0.0582 - val_accuracy: 0.9833
Epoch 2/20
540/540 [==============================] - 13s 24ms/step - loss: 0.0574 - accuracy: 0.9823 - val_loss: 0.0389 - val_accuracy: 0.9883
Epoch 3/20
540/540 [==============================] - 14s 26ms/step - loss: 0.0400 - accuracy: 0.9883 - val_loss: 0.0351 - val_accuracy: 0.9910
Epoch 4/20
540/540 [==============================] - 15s 28ms/step - loss: 0.0313 - accuracy: 0.9904 - val_loss: 0.0359 - val_accuracy: 0.9897
Epoch 5/20
540/540 [==============================] - 15s 28ms/step - loss: 0.0252 - accuracy: 0.9919 - val_loss: 0.0334 - val_accuracy: 0.9908
Epoch 6/20
540/540 [==============================] - 15s 28ms/step - loss: 0.0213 - accuracy: 0.9932 - val_loss: 0.0349 - val_accuracy: 0.9910
Epoch 7/20
540/540 [==============================] - 15s 27ms/step - loss: 0.0188 - accuracy: 0.9938 - val_loss: 0.0348 - val_accuracy: